In [1]:
import datetime as dt
from datetime import date
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import time
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

# https://www.youtube.com/watch?v=QIUxPv5PJOY

In [109]:
def format_data(df,start, end):
    training_data = df[start:end].drop('Date', axis = 1)
    return(training_data)
        

def create_xy(training_data, samples, forecast):
    x = []
    y = []
    L = len(training_data)
    samples = 100
    forecast = 20

    for i in range(samples, L - forecast):
        x.append(np.array(training_data[i-samples:i]))
        y.append(np.array(training_data[i+1:i+1+forecast]['pct chage price']))
    x = np.array(x)
    y = np.array(y)
    
    return([x,y])

#returns list of RSI values over n periods
def RSI(df, n):
    L = len(df)
    up = [0]*L
    down = [0]*L
    
    for i in range(1,L):
        if df['Close'][i-1] <= df['Close'][i]:
            up[i] = df['Close'][i] - df['Close'][i-1]
        if df['Close'][i-1] > df['Close'][i]:
            down[i] = abs(df['Close'][i] - df['Close'][i-1])

    avg_up= [0]*L
    avg_down = [0]*L
    
    avg_up[n] = np.mean(up[1:n+1])
    avg_down[n] = np.mean(down[1:n+1])
    
    for i in range(n+1,L):
        avg_up[i] = (avg_up[i-1]*(n-1)+up[i])/n
        avg_down[i] = (avg_down[i-1]*(n-1)+down[i])/n
    
    #return(avg_up,avg_down)
    RSI = [0]*L
    for i in range(n,L):
        RSI[i]= 100 - 100/(1+ (avg_up[i-1]/avg_down[i]))
    
    return(RSI)


##SMA, returns list of simple moving average over n periods
def SMA(stock_df,n):
    A = [0]*len(stock_df)
    for i in range(len(stock_df)-n+1):
        A[i+n-1] = np.mean(list(stock_df['Close'][i:i+n]))
    return(A)

#EMA, returns list of exponential moving average over n periods
def EMA(stock_df,n):
    mult = 2/(n+1)
    A = [0]*len(stock_df)
    A[n-1] = SMA(stock_df,n)[n-1]
    for i in range(n-1,len(stock_df)-1):
        A[i+1] = A[i]+(stock_df['Close'][i]-A[i])*mult
    return(A)

In [123]:
QQQ = pd.read_csv('dataset_QQQ_20140207_to_20181231.csv')
QQQ.drop('Date', axis =1, inplace = True)

In [124]:
QQQ_train = QQQ[0:1000].copy()
QQQ_test = QQQ[1000:].copy()

In [138]:
# predict loop test
QQQ = pd.read_csv('dataset_QQQ_20140207_to_20181231.csv')
QQQ.drop('Date', axis =1, inplace = True)
QQQ_test = QQQ[0:1000].copy()


def pred(input):
   # y_pred = model.predict
   # y_pred = y_pred/scaler.scale_[0]
    return np.sum(np.array(input[-60:])/60)

forecast = 50
for i in range(forecast):
    
    y_pred = pred(QQQ_test)
    
    new_row ={'Close':y_pred}
    QQQ_test.drop([' 9 RSI', '12 EMA', '26 EMA'], axis = 1, inplace = True)
    QQQ_test = QQQ_test.append(new_row, ignore_index = True,)
    QQQ_test[' 9 RSI'] = RSI(QQQ_test, 9)
    QQQ_test['12 EMA'] = EMA(QQQ_test, 12)
    QQQ_test['26 EMA'] = EMA(QQQ_test, 26)

In [155]:
def create_xy_scale(training_data, samples):
    scaler = MinMaxScaler()
    training_data = scaler.fit_transform(training_data)
    x = []
    y = []
    L = len(training_data)
    samples = 100
    forecast = 20

    for i in range(samples, L):
        x.append(training_data[i-samples:i])
        y.append(training_data[i, 0])
    x = np.array(x)
    y = np.array(y)
    s = scaler.scale_
    
    return([x,y,s])

In [170]:
QQQ = pd.read_csv('dataset_QQQ_20140207_to_20181231.csv')
QQQ.drop('Date', axis =1, inplace = True)
QQQ_train = QQQ[0:1000].copy()
x_train, y_train, scale = create_xy_scale(QQQ_train, 50)

array([ 3.19000244, 39.73351849,  0.        ,  0.        ])

In [171]:
QQQ_train

,Close,9 RSI,12 EMA,26 EMA
0,87.300003,49.544463,85.871403,86.778077
1,87.800003,59.932334,86.091188,86.816738
2,88.800003,62.751712,86.354082,86.889573
3,88.970001,67.842895,86.730378,87.031086
4,89.629997,68.662075,87.074935,87.174709
...,...,...,...,...
995,168.119995,85.108297,163.315003,160.648335
996,169.509995,88.496397,164.054233,161.201792
997,168.410004,79.882202,164.893581,161.817214
998,168.350006,79.315346,165.434569,162.305569


In [179]:
scaler = MinMaxScaler()
QQQ_train_scale = scaler.fit_transform(QQQ_train)

In [181]:
scaler.scale_

array([0.01151808, 0.01273465, 0.01249817, 0.01316218])

In [186]:
QQQ_train_scale[1]/scaler.scale_[0]

array([3.69000244e+00, 5.12185751e+01, 2.38486358e-01, 4.41797133e-02])

In [177]:
np.array(QQQ_train).shape

(1000, 4)